# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
client = MongoClient()

In [2]:
db = client.companies

In [3]:
db.list_collection_names()

['companies']

In [4]:
dataset = db.companies.find({})

In [5]:
dataset = list(dataset)

In [6]:
dataset[0].keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [42]:
babelgum = db.companies.find({'name':"Babelgum"},{"name":1})
babelgum = list(babelgum)

In [8]:
babelgum

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [44]:
companies_5000 = db.companies.find(
    {'number_of_employees':{"$gt":5000}}
).sort([("number_of_employees",1)]).limit(20)

In [45]:
companies_5000 = list(companies_5000)

In [46]:
len(list(companies_5000))

20

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [12]:
founded = db.companies.find(
    {'founded_year':{"$gte":2000, "$lte":2005}},
    {'name':1, 'founded_year':1}
)

In [13]:
founded = list(founded)

In [14]:
len(founded)

3734

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [15]:
fil = {'$and':[
    {'ipo.valuation_amount':{'$gt':100000000}},
    {'founded_year':{'$lt':2010}}
    ]}
project = {'name':1,'ipo.valuation_amount':1}
valuable_companies = db.companies.find(fil,project).sort([("ipo.valuation_amount",1)])

In [16]:
valuable_companies = list(valuable_companies)

In [17]:
valuable_companies[0:3]

[{'_id': ObjectId('52cdef7c4bab8bd67529823b'),
  'name': 'Salesforce',
  'ipo': {'valuation_amount': 110000000}},
 {'_id': ObjectId('52cdef7c4bab8bd6752987fe'),
  'name': 'Geeknet',
  'ipo': {'valuation_amount': 134000000}},
 {'_id': ObjectId('52cdef7c4bab8bd6752984e3'),
  'name': 'QuinStreet',
  'ipo': {'valuation_amount': 140000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [18]:
fil = {'number_of_employees':{'$lt':1000}, 'founded_year':{'$lt':2005}}
result = db.companies.find(fil).limit(10)

In [19]:
result = list(result)
len(result)

10

### 6. All the companies that don't include the `partners` field.

In [20]:
result = db.companies.find({'partners':{'$exists':'false'}})
result = list(result)
len(result)

18801

### 7. All the companies that have a null type of value on the `category_code` field.

In [21]:
result = db.companies.find({'category_code':{'$type':10}})
result = list(result)
len(result)

2751

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [22]:
result = db.companies.find(
    {'number_of_employees':{'$gte':100, '$lt':1000}},
    {'name':1, 'number_of_employees':1}
)
result = list(result)
len(result)

917

### 9. Order all the companies by their IPO price in a descending order.

In [23]:
result = db.companies.find({}).sort([("ipo.valuation_amount",-1)])
result = list(result)

In [24]:
result[0]['name'], result[0]['ipo']

('GREE',
 {'valuation_amount': 108960000000,
  'valuation_currency_code': 'JPY',
  'pub_year': 2008,
  'pub_month': 12,
  'pub_day': 17,
  'stock_symbol': '3632'})

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [25]:
result = db.companies.find(
    {},{'name':1, '_id':0}).sort([("number_of_employees",-1)]).limit(10)
result = list(result)
result

[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [26]:
result = db.companies.find({'founded_month':{'$gte':6}}).limit(1000)
result = list(result)
len(result)

1000

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [27]:
fil = {'$and':[
    {'acquisitions.price_amount':{'$gt':10000}},
    {'founded_year':{'$lt':2000}}
    ]}
result = db.companies.find(fil).sort('acquisitions.price_amount',-1)
result = list(result)
result[0]['name'], result[0]['acquisitions'][0]

('Sprint Nextel',
 {'price_amount': 35000000000,
  'price_currency_code': 'USD',
  'term_code': None,
  'source_url': 'http://www.theage.com.au/news/breaking/sprint-completes-nextel-acquisition/2005/08/15/1123957976577.html',
  'source_description': 'Sprint completes Nextel acquisition',
  'acquired_year': 2005,
  'acquired_month': 8,
  'acquired_day': 15,
  'company': {'name': 'Nextel Communications', 'permalink': 'nextel'}})

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [33]:
result = db.companies.find(
    {'acquisitions.acquired_year':{'$gt':2010}},
    {'name':1, 'acquisitions':1}).sort('acquisitions.price_amount',-1)
result = list(result)
result[0]

{'_id': ObjectId('52cdef7c4bab8bd6752986da'),
 'name': 'Sprint Nextel',
 'acquisitions': [{'price_amount': 35000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.theage.com.au/news/breaking/sprint-completes-nextel-acquisition/2005/08/15/1123957976577.html',
   'source_description': 'Sprint completes Nextel acquisition',
   'acquired_year': 2005,
   'acquired_month': 8,
   'acquired_day': 15,
   'company': {'name': 'Nextel Communications', 'permalink': 'nextel'}},
  {'price_amount': 483000000,
   'price_currency_code': 'USD',
   'term_code': 'stock',
   'source_url': 'http://www.foxbusiness.com/story/markets/industries/telecom/sprint-acquire-virgin-mobile--million/',
   'source_description': 'Sprint to Acquire Virgin Mobile for $483M',
   'acquired_year': 2009,
   'acquired_month': 7,
   'acquired_day': 28,
   'company': {'name': 'Virgin Mobile USA', 'permalink': 'virgin-mobile'}},
  {'price_amount': 831000000,
   'price_currency_code': 'USD',
 

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [39]:
result = db.companies.find({},
    {'name':1, 'founded_year':1, '_id':0}).sort('founded_year',-1)
result = list(result)
result[:5]

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [41]:
result = db.companies.find(
    {'acquisitions.acquired_day':{'$lte':7}}).sort('acquisitions.price_amount',-1)
result = list(result)
len(result)

42

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [51]:
fil = {'$and':[
    {'category_code':'web'},
    {'number_of_employees':{'$gt':4000}}
    ]}
result = db.companies.find(fil).sort('number_of_employees',1)
result = list(result)
len(result)

9

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [53]:
fil = {'$and':[
    {'acquisitions.price_amount':{'$gt':10000000}},
    {'acquisitions.price_currency_code':'EUR'}
    ]}
result = db.companies.find(fil)
result = list(result)
len(result)

17

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [62]:
result = db.companies.find(
    {'acquisitions.acquired_month':{'$lt':3}},
    {'name':1, 'acquisitions':1}).limit(10)
result = list(result)
len(result)

10

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code